In [6]:
#Path to Images
impath = "images_training_rev1/*.jpg"
#Path to Labels CSV
lapath = "training_solutions_rev1.csv"

In [7]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import skimage.io as io
import keras
from tqdm.notebook import trange,tqdm

%matplotlib inline

In [8]:
def lum(im):
    pixels = io.imread(im)
    return 0.21 * pixels[:,:,0] + 0.72 * pixels[:,:,1] + 0.07 * pixels[:,:,2]

In [9]:
class BatchManager:
    def __init__(self,galaxy_data,batches):
        self.galaxy_data = galaxy_data
        self.batches = batches
        self.idx = 0
        self.done = False
        self.size = self.batches.shape[1]
    
    def __call__(self):
        if self.done == True:
            return False
        label = np.take(self.galaxy_data.labels,self.batches[self.idx],axis=0)
        out = np.zeros((self.size,self.galaxy_data.imsize[0] * self.galaxy_data.imsize[1]))
        for i in range(self.size):
            out[i] = self.galaxy_data.image_collection[self.batches[self.idx][i]].flatten()/255
        self.idx += 1
        if self.idx  >= self.batches.shape[0]:
            self.done = True
        return label,out

class GalaxyData:
    def __init__(self,lapath=lapath,impath=impath,load_func=lum):
        self.impath,self.lapath,self.load_func = impath,lapath,load_func
        self.label_df = pd.read_csv(lapath,index_col=0)
        self.image_collection = io.ImageCollection(impath,load_func=load_func)
        self.labels = self.label_df.to_numpy()
        self.imsize = self.image_collection[0].shape
    
    def __call__(self,norm = 255):
        print("LOADING DATA")
        out = np.zeros((self.labels.shape[0],self.image_collection[0].shape[0] * self.image_collection[0].shape[0]))
        for i in trange(self.labels.shape[0]):
            out[i] = self.image_collection[i].flatten()/norm
        return self.labels,out
    
    def __getitem__(self,idx):
        label = self.labels[idx]
        im = self.image_collection[idx]
        return label,im
    
    def batch(self,size):
        indexes = np.arange(len(self.labels))
        np.random.shuffle(indexes)
        nims = indexes.shape[0]//size
        indexes = indexes[:nims*size]
        indexes = np.reshape(indexes,(nims,size))
        return BatchManager(self,indexes)

def display_image(im):
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.imshow(im, origin='lower', cmap='Greys')
    ax.set_aspect('auto')
    plt.show()

In [ ]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(424*424,)))
model.add(keras.layers.Dense(32))
model.add(keras.layers.Dense(32))
model.add(keras.layers.Dense(37))

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-20 18:06:09.730753: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-20 18:06:09.730959: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
